In [ ]:
pip install imblearn

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, roc_auc_score, roc_curve, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore")

In [13]:
df = pd.read_csv('/Users/roman/Desktop/Яндекс Практикум/Data Science/Datasets/Churn (1).csv')

In [14]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [15]:
df['Tenure'] = df['Tenure'].fillna(-1)
data = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1).copy()
data = pd.get_dummies(data, drop_first=True)

In [16]:
x = data.drop(['Exited'], axis=1)
y = data['Exited']

x_train, x_val, y_train, y_val = \
train_test_split(x, y, test_size=0.4, random_state=42)

x_val, x_test, y_val, y_test = \
train_test_split(x_val, y_val, test_size=0.5, random_state=42)

print(x_train.shape,
      x_val.shape,
      x_test.shape,
      y_train.shape,
      y_val.shape,
      y_test.shape, end='')

(6000, 11) (2000, 11) (2000, 11) (6000,) (2000,) (2000,)

In [17]:
dt_model = DecisionTreeClassifier(random_state=42)
dt_params = { 'max_depth': range (1,16,2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

In [18]:
dt_grid = GridSearchCV(dt_model, dt_params, cv=5)
dt_grid.fit(x_train, y_train)
dt_grid.best_params_

{'max_depth': 7, 'min_samples_leaf': 1, 'min_samples_split': 2}

In [19]:
dt_model = DecisionTreeClassifier(max_depth=7,
                                  min_samples_leaf=1,
                                  min_samples_split=2,
                                  random_state=42)
dt_model.fit(x_train, y_train)
dt_predict_val = dt_model.predict(x_val)
dt_f1 = f1_score(y_val, dt_predict_val)
print('Значение метрики F1 для решающего дерева составляет:', f'{dt_f1:.3f}')

Значение метрики F1 для решающего дерева составляет: 0.518


In [20]:
rf_model = RandomForestClassifier(random_state=42)
rf_params = { 'n_estimators': range (10, 51, 10),
              'max_depth': range (1,16, 2),
              'min_samples_leaf': range (1,8),
              'min_samples_split': range (2,10,2) }

In [10]:
rf_grid = GridSearchCV(rf_model, rf_params, cv=4)
rf_grid.fit(x_train, y_train)
rf_grid.best_params_
%%time

KeyboardInterrupt: 

In [27]:
best_result_rf = 0
best_est = 0
for est in range(2, 101, 2):
    rf_model = RandomForestClassifier(random_state=42, n_estimators=est)
    rf_model.fit(x_train, y_train)
    rf_predict = rf_model.predict(x_val)
    result = f1_score(y_val, rf_predict)
    if result > best_result_rf:
        best_result_rf = result
        best_est = est

print('F1 наилучшей модели на валидационной выборке:', f'{best_result_rf:.3f}')
print('Наилучший показатель достигнут при значении est:', best_est)

F1 наилучшей модели на валидационной выборке: 0.570
Наилучший показатель достигнут при значении est: 50
